In [1]:
import tensorflow as tf
from tensorflow import keras
from data_load import DataLoad

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
dl = DataLoad()
(ds_train, ds_test), ds_info = dl.load_tfds()

In [4]:
ds_info.features

FeaturesDict({
    'bbox': BBoxFeature(shape=(4,), dtype=tf.float32),
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=196),
})

In [5]:
#resnet을 base model로하는 CAM을 뽑을 수 있는 모델을 만들어 보자.
num_classes = ds_info.features["label"].num_classes
base_model = keras.applications.resnet.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3),
    pooling='avg'
)
x = base_model.output
preds = keras.layers.Dense(num_classes, activation="softmax")(x)
cam_model = keras.Model(inputs=base_model.input, outputs=preds)

In [6]:
ds_train_norm = dl.apply_normalize_on_dataset(ds_train)
ds_test_norm = dl.apply_normalize_on_dataset(ds_test)

In [7]:
tf.random.set_seed(2020)
cam_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01),
    metrics=['accuracy'],
)

In [8]:
import os
file_path = os.path.join(os.getenv("HOME"), "project/Going_deeper/GD_gradCAM/best_checkpoint.h5")
getbestmodel = keras.callbacks.ModelCheckpoint(file_path, verbose=1, save_best_only=True)

In [9]:
history_cam_model = cam_model.fit(
    ds_train_norm, #normalized 학습데이터 (x, y) 둘 다 가지고 있음
    steps_per_epoch = int(ds_info.splits["train"].num_examples/16), #batch size = 16
    validation_steps=int(ds_info.splits["test"].num_examples/16), #test의 16개를 Validation
    epochs=20,
    callbacks=[getbestmodel],
    validation_data=ds_test_norm, #validation dataset
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/20
509/509 [==============================] - ETA: 0s - loss: 4.1418 - accuracy: 0.1729
Epoch 00001: val_loss improved from inf to 5.43121, saving model to /home/aiffel0042/project/Going_deeper/GD_gradCAM/best_checkpoint.h5
509/509 [==============================] - 91s 179ms/step - loss: 4.1418 - accuracy: 0.1729 - val_loss: 5.4312 - val_accuracy: 0.0069
Epoch 2/20
509/509 [==============================] - ETA: 0s - loss: 1.6170 - accuracy: 0.6548
Epoch 00002: val_loss improved from 5.43121 to 1.86947, saving model to /home/aiffel0042/project/Going_deeper/GD_gradCAM/best_checkpoint.h5
509/509 [==============================] - 80s 158ms/step - loss: 1.6170 - accuracy: 0.6548 - val_loss: 1.8695 - val_accuracy: 0.5108
Epoch 3/20
509/509 [==============================] - ETA: 0s - loss: 0.5163 - accuracy: 0.9110
Epoch 00003: val_loss improved from 1.86947 to 1.27520, saving model to /home/aiffel0042/project/Going_deeper/GD_gradCAM/best_checkpoint.h5
509/509 [===================

In [10]:
# model save하기
cam_model_path = os.getenv("HOME")+'/aiffel/class_activation_map/cam_model.h5'
cam_model.save(cam_model_path)